In [1]:
# import statements
import numpy as np
import random
from Monte_Carlo_without_es import Monte_carlo_without_es
from Sarsa import Sarsa
from Q_learning import Q_learning

In [2]:
# four by four obstacle
four_by_four_obs = [(1,1), (1,3), (2,3), (3,0)]

## Monte Carlo without Exploring Start

### Code

Usage: Create a Monte_carlo_without_es object, and call the generate_path() method.

### 4 x 4 Grid

In [3]:
mont1 = Monte_carlo_without_es(4, 4, obstacle_pos=four_by_four_obs)
print(mont1.get_map())
mont1.generate_path()
print(mont1.get_path_map())

|            |
|    X     X |
|          X |
| X        G |
| O          |
| O  X     X |
| O  O  O  X |
| X     O  G |


### 10 x 10 Grid

In [4]:
mont2 = Monte_carlo_without_es(10, 10)
print(mont2.get_map())
mont2.generate_path()
print(mont2.get_path_map())

|                         X    |
|       X                      |
|             X           X    |
|    X              X          |
| X              X     X  X    |
|    X           X  X  X       |
|    X              X          |
|    X  X     X                |
|          X        X  X       |
|       X  X              X  G |


KeyboardInterrupt: 

## Sarsa

### Code

Usage: Create a Sarsa object, and call the generate_path() method.

### 4 x 4 Grid

In [5]:
sarsa1 = Sarsa(4, 4, obstacle_pos=four_by_four_obs)
print(sarsa1.get_map())
sarsa1.generate_path()
print(sarsa1.get_path_map())

|            |
|    X     X |
|          X |
| X        G |
| O  O  O    |
|    X  O  X |
|       O  X |
| X     O  G |


### 10 x 10 Grid

In [6]:
sarsa2 = Sarsa(10, 10)
print(sarsa2.get_map())
sarsa2.generate_path()
print(sarsa2.get_path_map())


|                   X  X       |
|       X     X        X       |
|                            X |
|             X     X  X       |
| X  X     X                 X |
| X        X              X    |
| X        X              X  X |
|                      X  X    |
| X        X                   |
|    X                       G |


KeyboardInterrupt: 

## Q-Learning

### Code

Usage: Create a Q_learning object, and call the generate_path() method.

### 4 x 4 Grid

In [7]:
q_learn1 = Q_learning(4, 4, obstacle_pos=four_by_four_obs)
print(q_learn1.get_map())
q_learn1.generate_path()
print(q_learn1.get_path_map())

|            |
|    X     X |
|          X |
| X        G |
| O  O  O    |
|    X  O  X |
|       O  X |
| X     O  G |


### 10 x 10 Grid

In [8]:
q_learn2 = Q_learning(10, 10)
print(q_learn2.get_map())
q_learn2.generate_path()
print(q_learn2.get_path_map())

|                   X  X       |
|       X     X        X       |
|                            X |
|             X     X  X       |
| X  X     X                 X |
| X        X              X    |
| X        X              X  X |
|                      X  X    |
| X        X                   |
|    X                       G |


KeyboardInterrupt: 